## 인공지능 자동 분류기 개발 하기

In [1]:
# 필요한 라이브러리 불러오기

import cv2
import numpy as np
import time
import serial
import serial.tools.list_ports
import tensorflow.keras
from tensorflow.keras.models import model_from_json

In [2]:
# 마이크로비트 연결 com 포트 찾기

ports = serial.tools.list_ports.comports()
com = ''
for port, desc, hwid in sorted(ports):       
    if 'USB' in desc:
        com = port
if com != '':
    print('\n microbit USB detected: ', com)
else:
    print('\nPlease connect your microbit to this PC via USB')

ser = serial.Serial(com, 115200, timeout=0, parity=serial.PARITY_NONE, rtscts=0)


 microbit USB detected:  COM9


In [3]:
def SerialSendCommand(cmd):
    cmd = str(cmd) 
    cmd  = cmd + '\n'
    cmd = str.encode(cmd) 
    ser.write(cmd)  

In [9]:
# servo motor align
align = 85
SerialSendCommand(align)

# set crop
top = .55
bottom = .25

In [5]:
# 라벨 불러오기
labels = []
file = open("labels.txt", "r")
for x in file:
  labels.append(x.rstrip('\n'))
label_count = len(labels)
print('labels = ', labels)
file.close()

# 인공지능 모델 불러오기
model = tensorflow.keras.models.load_model('keras_model.h5', compile = False)

labels =  ['0 Blue', '1 Green', '2 None']


In [6]:
def ReadPicture(cropped, model, frame):
    
    score = 0
    scoreLabel = 0 
    
    # 구글 teachable Machine 'Export your model to use it in projects' 코드 참조 시작
    input_width = 224
    input_height = 224
    resized_image = cv2.resize(cropped, (input_width, input_height))
    imgRGB =cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)
    
    image_array = np.asarray(imgRGB)    
    normalized_image = (image_array.astype(np.float32) / 127.0) - 1
    batch = normalized_image.reshape(1, input_height, input_width, 3)
    prediction = model.predict(batch)
    # 구글 teachable Machine 'Export your model to use it in projects' 코드 참조 끝
    
    for x in range(0, label_count):
        if score < prediction[0][x]:
            scoreLabel = x
            score = prediction[0][x]
    return scoreLabel, score

In [7]:
 def Main():
        
    MODE_PAUSE = -1 
    MODE_START = 0

    mode_status = MODE_PAUSE
  
    video_capture = cv2.VideoCapture(0)
    height = video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT )
    width = video_capture.get(cv2.CAP_PROP_FRAME_WIDTH )

    y1 = int(height - (height * top))
    y2 = int(height - (height * bottom))
    square = (y2-y1)/2
    x1 = int(width/2 - square)
    x2 = int(width/2 + square)

    tic = int(round(time.time() * 1000)) + 1500

    print("Press [q] to quit") 
    print("Press [p] to start/pause")
    
    cmd = align
    SerialSendCommand(align)
    
    while(True):

        grabbed, frame = video_capture.read()
        cropped = frame[y1:y2, x1:x2]

        # rectangle around the cropped area
        rectangleColor              = (0, 255, 0)
        rectangleThickness          = 2
        cv2.rectangle(frame, (x1,y1), (x2,y2), rectangleColor, rectangleThickness)

        if mode_status == MODE_START:

            rectangleColor            = (255,255,255)
            cv2.rectangle(frame, (x1,y1), (x2,y2), rectangleColor, rectangleThickness)
       
            # Every 1 seconds, send prediction to micro:bit
            now = int(round(time.time() * 1000))

            #run the sorting algorithm
            scoreLabel, score = ReadPicture(cropped, model, frame)    
        
            if cmd == align and now > tic:
              
                if scoreLabel == 0 and score > .85:
                    cmd = align + 75
                    print('Send:', labels[scoreLabel])
                    tic = int(round(time.time() * 1000)) + 1000
                    SerialSendCommand(cmd)

                elif scoreLabel == 1 and score > .85:
                    cmd = align - 75
                    print('Send:', labels[scoreLabel])
                    tic = int(round(time.time() * 1000)) + 1000
                    SerialSendCommand(cmd)
                    
                elif scoreLabel == 2:
                    pass

            elif now > tic and cmd != align:
                cmd = align
                tic = int(round(time.time() * 1000)) + 1000
                SerialSendCommand(cmd)
            

        # 결과 영상 보기
        cv2.imshow("Frame", frame) 
        key = cv2.waitKey(1) & 0xFF

        # 제어문 키보드로 제어하기
        if key == ord("q"):
            print('Mode: Quit')
            break

        elif key == ord('p'):
            if mode_status != MODE_PAUSE:
                mode_status = MODE_PAUSE       
                print("Mode: Paused")
                SerialSendCommand(align)
            elif mode_status == MODE_PAUSE:
                mode_status = MODE_START
                print ("Mode: Start")

    video_capture.release()
    cv2.destroyAllWindows()


In [10]:
# 프로그램 실행

if __name__ == '__main__': 
    Main()

Press [q] to quit
Press [p] to start/pause
Mode: Start
Send: 0 Blue
Send: 1 Green
Send: 1 Green
Mode: Quit
